In [1]:
# Get all ProcedureResources related to Reporting
# rep.reporting_resources.all()

In [5]:
from openpyxl import load_workbook
from helpers.dates import parse_dict_dates, to_utc_string
from apps.service_orders.serializers import ProcedureResourceSerializer
from helpers.strings import (
    COMMON_IMAGE_TYPE,
    clean_latin_string,
    dict_to_casing,
    get_obj_from_path,
    keys_to_snake_case,
    to_snake_case,
)
import datetime
import uuid
from tqdm.notebook import tqdm

In [6]:
company = Company.objects.get(name='Arteris Intervias')
user=User.objects.get(username='rlcs')
company

<Company: b8450161-14d1-4de8-b254-7d1ddc4a9c20: Arteris Intervias>

In [7]:
filename='Intervias planilha de dados para lançar'

In [8]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[0]
# sheetname
ws = wb[sheetname]

In [9]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)

In [10]:
values[45]

{'Serial': 'INT-SIN-Aer-2023.03743',
 'Rodovia': 'SP-147',
 'km inicial': 114.19,
 'km final': 114.19,
 'Lote': '',
 'Latitude': -22.56704336467393,
 'Longitude': -47.437752378862214,
 'Natureza': 'Sinalização vertical',
 'Classe': 'Reposição de demais placas',
 'Empresa': 'IASIN - SINALIZAÇÕES',
 'Equipe': 'SIN.VERTICAL-01_IASIN_26228 - (EQUIPE02-MAURO)',
 'Programação': 'IASIN SV - OS.249/23 (KCOR 02719-23/Marcador de Alinhamento)',
 'Valor Total': 248.16276390000002,
 'Recurso_1': '15.10 - Retirada de Placa - Placa de solo - Marcadores de Alinhamento padrão DER/SP (MA) (m²)',
 'Quantidade_1': 0.3,
 'Valor_1': 44.4080736,
 'Valor Unitário_1': 148.026912,
 'Recurso_2': '5.6 - Fornecimento e Implantação em solo de placas indicativas, educativas, serviços, dispositivos de segurança etc. confeccionadas em chapa de aço galvanizado, com película tipo X, exceto os elementos em preto, que deverão ser não refletivos com película tipo IV. - Marcador de Obstáculo com pelicula tipo X (m²)',
 'Qu

In [55]:
resources_list=[]
for index,a in enumerate(tqdm(values)):
    rep = Reporting.objects.get(number=a['Serial'],company=company)
    resources = Resource.objects.filter(
                company=company,
                resource_service_orders__contract__extra_info__r_c_number='26228',
                resource_service_orders__contract__unit_price_services__firms=rep.firm,
            ).distinct()
    
    for i in range(1,11):
        recurso='Recurso_'+str(i)
        quantidade = 'Quantidade_'+str(i)
        if a.get(recurso):
            
            r=a[recurso].replace(a[recurso].split(' ')[-1],'').strip()
#             r=' '.join(d for index,d in enumerate(a[recurso].split(' ')) if index != 0)
            quantity = a[quantidade]
            if quantity:
                try:    
                    found_resource = resources.filter(
                                    name__icontains=r.lower())
                    if not found_resource:
                        found_resource = resources.filter(
                                    name__icontains=r[:200].lower())[0]
                    else:
                        found_resource = found_resource[0]
                    so_resource = found_resource.resource_service_orders.all()[0]
                except Exception as e:
                    print(r,e,index+2)
                    print(fg.ju)
                else:
                    unit_price = so_resource.unit_price
                    resources_list.append(
                        {
                            "uuid": str(uuid.uuid4()),
                            "total_price": quantity * unit_price,
                            "unit_price": unit_price,
                            "amount": quantity,
                            "reporting_id": rep.uuid,
                            "resource_id": str(found_resource.uuid),
                            "service_order_resource_id": str(so_resource.uuid),
                            "created_by_id": user.uuid,
                            "creation_date": datetime.datetime.now(),
                            "firm_id": str(rep.firm.uuid),
                        }
                    )
            else:
                print('Quantidade zerada: linha', index+2)

  0%|          | 0/59 [00:00<?, ?it/s]

In [58]:
len(resources_list),resources_list

(116,
 [{'uuid': 'f0f5c5ac-b933-4e8f-8da5-96991b1a08cf',
   'total_price': 203.75469030000002,
   'unit_price': 679.182301,
   'amount': 0.3,
   'reporting_id': UUID('663543ae-c616-4137-99bf-b605b97be813'),
   'resource_id': '3919909c-0fcb-4353-8d4d-da4f83aafb76',
   'service_order_resource_id': '5c3f0d86-fcf0-49b4-8d07-86b4d219da67',
   'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
   'creation_date': datetime.datetime(2024, 5, 9, 11, 28, 51, 586293),
   'firm_id': '9dcb5d25-8304-4ccd-8621-9627ec082d2f'},
  {'uuid': 'b7a0cce7-3608-4ab1-874a-900fff0e677e',
   'total_price': 44.4080736,
   'unit_price': 148.026912,
   'amount': 0.3,
   'reporting_id': UUID('663543ae-c616-4137-99bf-b605b97be813'),
   'resource_id': '97fbe417-c9b2-48f9-9d2e-eead8536e467',
   'service_order_resource_id': '48dcb761-5006-41af-a2fc-823e856c5de4',
   'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
   'creation_date': datetime.datetime(2024, 5, 9, 11, 28, 51, 593710),
   'firm_id

In [62]:
for resource in tqdm(resources_list):
    pr=ProcedureResource(**resource)
    pr.save()

  0%|          | 0/116 [00:00<?, ?it/s]

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField ProcedureResource.creation_date received a naive datetime (2024-05-09 11:28:51.586293) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField HistoricalProcedureResource.creation_date received a naive datetime (2024-05-09 11:28:51.586293) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


unhashable type: 'list'
unhashable type: 'list'


/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField ProcedureResource.creation_date received a naive datetime (2024-05-09 11:28:51.593710) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField HistoricalProcedureResource.creation_date received a naive datetime (2024-05-09 11:28:51.593710) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField ProcedureResource.creation_date received a naive datetime (2024-05-09 11:28:51.609612) while time zone support is active.
  warnings.warn

unhashable type: 'list'
unhashable type: 'list'
unhashable type: 'list'


...

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField ProcedureResource.creation_date received a naive datetime (2024-05-09 11:28:52.777896) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField HistoricalProcedureResource.creation_date received a naive datetime (2024-05-09 11:28:52.777896) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField ProcedureResource.creation_date received a naive datetime (2024-05-09 11:28:52.785017) while time zone support is active.
  warnings.warn

unhashable type: 'list'
unhashable type: 'list'
unhashable type: 'list'
unhashable type: 'list'
unhashable type: 'list'
unhashable type: 'list'
unhashable type: 'list'
unhashable type: 'list'
unhashable type: 'list'


/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField HistoricalProcedureResource.creation_date received a naive datetime (2024-05-09 11:28:52.868134) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField ProcedureResource.creation_date received a naive datetime (2024-05-09 11:28:52.875293) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField HistoricalProcedureResource.creation_date received a naive datetime (2024-05-09 11:28:52.875293) while time zone support is active.
  war

unhashable type: 'list'
unhashable type: 'list'
unhashable type: 'list'
unhashable type: 'list'
unhashable type: 'list'
unhashable type: 'list'
unhashable type: 'list'
unhashable type: 'list'
unhashable type: 'list'
unhashable type: 'list'


/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField HistoricalProcedureResource.creation_date received a naive datetime (2024-05-09 11:28:52.995669) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField ProcedureResource.creation_date received a naive datetime (2024-05-09 11:28:53.021328) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField HistoricalProcedureResource.creation_date received a naive datetime (2024-05-09 11:28:53.021328) while time zone support is active.
  war

In [11]:
pr.reporting.number

'APS-2023.08510'

In [12]:
values[-1]

{'': None,
 'Serial': 'APS-2023.08510',
 'Rodovia': 'BR116-SC',
 'km inicial': 179.16,
 'km final': 179.1591,
 'Lote': 'Lote 2',
 'Latitude': -27.262055005507168,
 'Longitude': -50.44598820838514,
 'Natureza': 'Sinalização vertical',
 'Classe': 'Implantação (solo/aérea) de Sinalização vertical',
 'Comprimento': 0.9,
 'Largura': 0.3,
 'Espessura': 0,
 'Faixa': 'Pista de serviço',
 'Sentido': 'Norte',
 'Status': 'Executado',
 'Criado por': 'Graziele Alves',
 'Atualizado por': 'Graziele Alves',
 'Empresa': 'JC Sinalização Ltda',
 'Equipe': 'JC Sinalização Ltda',
 'Programação': 'Oficio SEI 33712_Solic 24.10.23',
 'Início da Programação': datetime.datetime(2023, 10, 24, 17, 53, 17, 484000),
 'Fim da Programação': datetime.datetime(2023, 12, 8, 17, 52),
 'Criado em': datetime.datetime(2023, 10, 24, 17, 38, 12),
 'Encontrado em': datetime.datetime(2023, 10, 24, 17, 38, 12),
 'Atualizado em': datetime.datetime(2024, 4, 1, 14, 43, 30, 521000),
 'Executado em': datetime.datetime(2023, 11, 22, 0

# Check Recursos salvos nos apontamentos

In [63]:
total_apt=0
total_plan=0
for a in tqdm(values):
    rep = Reporting.objects.get(number=a['Serial'],company=company)
    counter=0
    for i in range(1,11):
        recurso='Recurso_'+str(i)
        if a.get(recurso):
            counter+=1
            total_plan+=1
    total_apt+=rep.reporting_resources.all().count()
    print(rep.number,rep.reporting_resources.all().count(),counter)
print(total_plan,total_apt)

  0%|          | 0/59 [00:00<?, ?it/s]

INT-SIN-Aer-2023.03618 4 2
INT-SIN-Aer-2023.03619 4 2
INT-SIN-Aer-2023.03620 4 2
INT-SIN-Aer-2023.03621 4 2
INT-SIN-Aer-2023.03622 4 2
INT-SIN-Aer-2023.03623 4 2
INT-SIN-Aer-2023.03624 4 2
INT-SIN-Aer-2023.03625 4 2
INT-SIN-Aer-2023.03626 4 2
INT-SIN-Aer-2023.03627 4 2
INT-SIN-Aer-2023.03628 4 2
INT-SIN-Aer-2023.03629 4 2
INT-SIN-Aer-2023.03630 4 2
INT-SIN-Aer-2023.03631 4 2
INT-SIN-Aer-2023.03632 4 2
INT-SIN-Aer-2023.03633 4 2
INT-SIN-Aer-2023.03634 4 2
INT-SIN-Aer-2023.03635 4 2
INT-SIN-Aer-2023.03636 4 2
INT-SIN-Aer-2023.03637 4 2
INT-SIN-Aer-2023.03638 4 2
INT-SIN-Aer-2023.03639 4 2
INT-SIN-Aer-2023.03707 4 2
INT-SIN-Aer-2023.03708 4 2
INT-SIN-Aer-2023.03709 4 2
INT-SIN-Aer-2023.03710 4 2
INT-SIN-Aer-2023.03711 4 2
INT-SIN-Aer-2023.03712 4 2
INT-SIN-Aer-2023.03713 4 2
INT-SIN-Aer-2023.03714 4 2
INT-SIN-Aer-2023.03715 4 2
INT-SIN-Aer-2023.03716 4 2
INT-SIN-Aer-2023.03717 4 2
INT-SIN-Aer-2023.03718 4 2
INT-SIN-Aer-2023.03719 4 2
INT-SIN-Aer-2023.03720 4 2
INT-SIN-Aer-2023.03721 4 2
I

In [64]:
rep.reporting_resources.first().__dict__

{'_state': <django.db.models.base.ModelState at 0x74b55e8781c0>,
 'uuid': UUID('3e64ed4d-123a-455b-af75-38cf6f2f9628'),
 'procedure_id': None,
 'service_order_id': None,
 'resource_id': UUID('ac75cc09-428f-43ab-ad20-a784f696b58c'),
 'service_order_resource_id': UUID('b748341d-8bd4-4537-bf3a-c7325ddfa61f'),
 'created_by_id': UUID('552e85a0-e226-4e47-8c85-f9d03ee9bddd'),
 'amount': 8.0,
 'unit_price': 676.0,
 'total_price': 5408.0,
 'creation_date': datetime.datetime(2023, 8, 24, 12, 49, 35, tzinfo=<UTC>),
 'approval_status': 'APPROVED_APPROVAL',
 'approval_date': datetime.datetime(2024, 3, 18, 19, 1, 16, 35036, tzinfo=<UTC>),
 'approved_by_id': UUID('1b107e99-d646-44da-b712-629599ec6bdb'),
 'measurement_bulletin_id': UUID('064dac0f-46f1-4640-a313-762cf6397793'),
 'firm_id': UUID('b2e661f1-7a83-4e23-8ea9-293de1045a2d'),
 'reporting_id': UUID('1c05edf6-5b99-40b0-9534-1c6f45d833c7'),
 '_fieldsignals_originals': {(128322821116496,
   128322354446400): {'uuid': UUID('3e64ed4d-123a-455b-af75-

# Deletar recursos antigos do contrato, pois foi adicionado novos

In [67]:
for a in tqdm(values):
    rep = Reporting.objects.get(number=a['Serial'],company=company)
    rep.reporting_resources.filter(creation_date__lte='2024-04-30').delete()

  0%|          | 0/59 [00:00<?, ?it/s]

# Check Equipes do contrato

In [21]:
for index,a in enumerate(tqdm(values)):
    rep = Reporting.objects.get(number=a['Serial'],company=company)
    print(index+2,rep.firm.name)

  0%|          | 0/59 [00:00<?, ?it/s]

2 SIN.VERTICAL-01_IASIN_26228
3 SIN.VERTICAL-01_IASIN_26228
4 SIN.VERTICAL-01_IASIN_26228
5 SIN.VERTICAL-01_IASIN_26228
6 SIN.VERTICAL-01_IASIN_26228
7 SIN.VERTICAL-01_IASIN_26228
8 SIN.VERTICAL-01_IASIN_26228
9 SIN.VERTICAL-01_IASIN_26228
10 SIN.VERTICAL-01_IASIN_26228
11 SIN.VERTICAL-01_IASIN_26228
12 SIN.VERTICAL-01_IASIN_26228
13 SIN.VERTICAL-01_IASIN_26228
14 SIN.VERTICAL-01_IASIN_26228
15 SIN.VERTICAL-01_IASIN_26228
16 SIN.VERTICAL-01_IASIN_26228
17 SIN.VERTICAL-01_IASIN_26228
18 SIN.VERTICAL-01_IASIN_26228
19 SIN.VERTICAL-01_IASIN_26228
20 SIN.VERTICAL-01_IASIN_26228
21 SIN.VERTICAL-01_IASIN_26228
22 SIN.VERTICAL-01_IASIN_26228
23 SIN.VERTICAL-01_IASIN_26228
24 SIN.VERTICAL-01_IASIN_26228
25 SIN.VERTICAL-01_IASIN_26228
26 SIN.VERTICAL-01_IASIN_26228
27 SIN.VERTICAL-01_IASIN_26228
28 SIN.VERTICAL-01_IASIN_26228
29 SIN.VERTICAL-01_IASIN_26228
30 SIN.VERTICAL-01_IASIN_26228
31 SIN.VERTICAL-01_IASIN_26228
32 SIN.VERTICAL-01_IASIN_26228
33 SIN.VERTICAL-01_IASIN_26228
34 SIN.VERTICAL-